In [2]:
import gym
from rich import print

## list all the atari gym environments
env_names = [env_spec.id for env_spec in gym.envs.registry.all()]
# print( env_names )
## print all the envs name with atari

In [3]:
env_id = "BreakoutNoFrameskip-v4"
seed = 42
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk
envs = gym.vector.SyncVectorEnv(
        [make_env( env_id = env_id, 
                  seed = seed , idx = 42, capture_video = False , run_name = "start") for i in range(1)]

)


/home/null/miniconda3/envs/clean-rl/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: module 'gym.envs.atari' has no attribute 'AtariEnv'

In [ ]:
## Get the Agent from ppo atari
from cleanrl.ppo_atari import Agent

agent = Agent(envs)

/home/null/miniconda3/envs/clean-rl/lib/python3.7/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


In [ ]:
next_obs = envs.reset()
print( 5 )
print( next_obs.shape)

/home/null/miniconda3/envs/clean-rl/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


5

(1, 210, 160, 3)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stable_baselines3.common.buffers import ReplayBuffer
model = nn.Sequential(
    nn.Linear( 5 , 5),
    nn.ReLU(),
    nn.Linear( 5 , 10 ))
from rich import print
print( model )

/home/null/miniconda3/envs/clean-rl/lib/python3.7/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Sequential(
  (0): Linear(in_features=5, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=10, bias=True)
)

In [7]:
print( model[0])
print( model[0].weight.data)
print( model[0].bias.data)

/home/null/miniconda3/envs/clean-rl/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Linear(in_features=5, out_features=5, bias=True)

tensor([[-0.0631, -0.3211,  0.1332, -0.3825, -0.4366],
        [ 0.2213, -0.4126,  0.0538, -0.2671,  0.2106],
        [-0.3278, -0.1579,  0.4328, -0.2887, -0.3082],
        [ 0.2555,  0.3162, -0.0125, -0.1352,  0.4457],
        [-0.0104, -0.0569, -0.3641, -0.4115,  0.4292]])

tensor([ 0.3335,  0.1415, -0.3759,  0.2956,  0.3593])

In [8]:
import torch

print( getattr( torch ,  "quint8"))
print( getattr( torch.ao.quantization ,  "MovingAverageMinMaxObserver"))

torch.quint8

<class 'torch.ao.quantization.observer.MovingAverageMinMaxObserver'>

In [9]:
model = nn.Sequential(
    torch.ao.quantization.QuantStub(),
    nn.Linear( 5 , 5),
    nn.ReLU(),
    nn.Linear( 5 , 10 ), 
    torch.ao.quantization.DeQuantStub())
print( model)

model.qconfig = torch.ao.quantization.get_default_qat_qconfig("fbgemm")

torch.ao.quantization.prepare_qat(model , inplace = True   )

print( model )
    

Sequential(
  (0): QuantStub()
  (1): Linear(in_features=5, out_features=5, bias=True)
  (2): ReLU()
  (3): Linear(in_features=5, out_features=10, bias=True)
  (4): DeQuantStub()
)

/home/null/miniconda3/envs/clean-rl/lib/python3.7/site-packages/torch/ao/quantization/observer.py:179: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


Sequential(
  (0): QuantStub(
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), 
zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (1): Linear(
    in_features=5, out_features=5, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), 
zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, 
qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), 
max_val=tensor([]))
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), 
zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (2): ReLU()
  (3): Linear(
    in_features=5, out_features=10, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), 
zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, 
qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), 
max_val=tensor([]))
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), 
zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (4): DeQuantStub()
)

In [10]:
x = torch.randn([1, 1, 5])
print(  x ) 
print( model(x))
print( model)

tensor([[[ 0.6523,  2.2140, -0.5017,  1.4603, -0.3738]]])

tensor([[[-0.5206, -0.2502, -0.2840, -0.0068,  0.3381, -0.2907, -0.2164,
          -0.1149,  0.1758,  0.2772]]],
       grad_fn=<FusedMovingAvgObsFqHelperBackward0>)

Sequential(
  (0): QuantStub(
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0214]), 
zero_point=tensor([23], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=-0.5017282366752625, 
max_val=2.213959217071533)
    )
  )
  (1): Linear(
    in_features=5, out_features=5, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0021, 
0.0013, 0.0032, 0.0029, 0.0022]), zero_point=tensor([0, 0, 0, 0, 0], dtype=torch.int32), 
dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, 
reduce_range=False
      (activation_post_process): 
MovingAveragePerChannelMinMaxObserver(min_val=tensor([-0.2691, -0.1546, -0.4061, -0.3704, 
-0.2764]), max_val=tensor([0.0888, 0.1597, 0.0393, 0.3261, 0.2842]))
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0107]), 
zero_point=tensor([56], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=-0.6004877686500549, 
max_val=0.7632002830505371)
    )
  )
  (2): ReLU()
  (3): Linear(
    in_features=5, out_features=10, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0016, 
0.0030, 0.0033, 0.0026, 0.0013, 0.0035, 0.0014, 0.0025, 0.0032,
              0.0032]), zero_point=tensor([   0,    0,    0,    0, -128,    0, -128,    0,   
0,    0],
             dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, 
qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(
        min_val=tensor([-0.2056, -0.3841, -0.3757, -0.2297,  0.1280, -0.4425,  0.0414, 
-0.3261,
                -0.4155, -0.4076]), max_val=tensor([0.1524, 0.1349, 0.4202, 0.3359, 0.3244, 
0.1962, 0.3674, 0.2940, 0.1837,
                0.3922])
      )
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0068]), 
zero_point=tensor([77], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=-0.5184717178344727, 
max_val=0.3402331471443176)
    )
  )
  (4): DeQuantStub()
)

In [11]:
print( model[0])

QuantStub(
  (activation_post_process): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0214]), 
zero_point=tensor([23], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, 
qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=-0.5017282366752625, 
max_val=2.213959217071533)
  )
)

In [12]:
y = model[0](x)
print( y.dtype)
y = model[1](y)
print( y.dtype)
y  = model[2](y)
print( y.dtype)

torch.float32

torch.float32

torch.float32